<a name='1'></a>
## Import Packages and load data

In [1]:
### -*- coding: utf-8 -*-
"""
Created on Mon May 24 13:26:13 2021

@author: kjsanche


"""

import time
#time.sleep(60*60*4)
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib as mpl
from itertools import compress
import numpy as np
import gc
import struct
import os
import glob
import random
import math
from twilio.rest import Client
#from decouple import config
from UNET_Functions import unet_model, summary
from Sat_contrail_read import Extract_RawDef, extract_img, extract_mask, extract_imglist, get_model_memory_usage
import tensorflow as tf
import sys
sys.path.append('/home/kjsanche/Desktop/Projects/loss')
#from loss_function import *
from tensorflow.python.ops.metrics_impl import false_positives, false_negatives
import tensorflow.keras.metrics as tfm
#import tensorflow_addons as tfa
from tensorflow.keras import backend as K
from tensorflow import keras
from tensorflow.keras import layers
#from focal_loss import BinaryFocalLoss, SparseCategoricalFocalLoss

configproto = tf.compat.v1.ConfigProto() 
#configproto.gpu_options.per_process_gpu_memory_fraction = 0.8 # fraction of memory
configproto.gpu_options.visible_device_list = "0"
sess = tf.compat.v1.Session(config=configproto) 
tf.compat.v1.keras.backend.set_session(sess)

sys_details = tf.sysconfig.get_build_info()
print(sys_details)
cudnn_version = sys_details["cudnn_version"]
cuda_version = sys_details["cuda_version"]

print('cuda version: ', cuda_version)
print('cudNN version: ',cudnn_version)
print('TF version: ', tf.version.VERSION)




BATCH_SIZE = 32
EPOCHS = 100
IMG_W=64
IMG_H=64
N_CHANNELS = 7
N_FILTERS = 32
LEARNING_RATE = 0.01
AUTO = tf.data.AUTOTUNE # used in tf.data.Dataset API
VERSION = 'v24' # v## or T121 for one case
TFrecord_path ='/home/kjsanche/Desktop/TFrecords/'
Models_path ='/home/kjsanche/Desktop/ExternalSSD/SatContrailData/Models/'

training_filenames=sorted(tf.io.gfile.glob([TFrecord_path + '*' + VERSION + '.tfrecords']))

if N_CHANNELS == 3:
    #validation_filenames=sorted(tf.io.gfile.glob([TFrecord_path + '*vVALIDATION.tfrecords']))
    validation_filenames=sorted(tf.io.gfile.glob([TFrecord_path + '*v3channel_val_novalidChannel.tfrecords']))#for v27
elif N_CHANNELS == 7:
    validation_filenames=sorted(tf.io.gfile.glob([TFrecord_path + '*v7CHANNEL_' +str(IMG_W)+ 'Val.tfrecords']))
#test_filenames=sorted(tf.io.gfile.glob([TFrecord_path + '*vTEST.tfrecords']))

random.Random(5).shuffle(training_filenames)
random.Random(5).shuffle(validation_filenames)

print(len(training_filenames))
print(len(validation_filenames))

2022-07-15 15:35:38.870446: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-07-15 15:35:39.909036: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-15 15:35:39.909742: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-15 15:35:39.916170: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-15 15:35:39.946627: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:21:00.0 name: Quadro P2000 computeCapability: 6.1
coreClock: 1.4805GHz coreC


OrderedDict([('cpu_compiler', '/home/builder/ktietz/aggregate/tensorflow_recipes/ci_cpu/tensorflow-base_1614583966145/_build_env/bin/x86_64-conda_cos6-linux-gnu-gcc'), ('cuda_compute_capabilities', ['compute_35', 'compute_52', 'compute_60', 'compute_61', 'compute_70', 'compute_75']), ('cuda_version', '10.1'), ('cudnn_version', '7'), ('is_cuda_build', True), ('is_rocm_build', False)])
cuda version:  10.1
cudNN version:  7
TF version:  2.4.1
2332
415


2022-07-15 15:35:40.767578: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1261] Device interconnect StreamExecutor with strength 1 edge matrix:
2022-07-15 15:35:40.767626: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1267]      0 
2022-07-15 15:35:40.767636: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1280] 0:   N 
2022-07-15 15:35:40.768281: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1406] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 4474 MB memory) -> physical GPU (device: 0, name: Quadro P2000, pci bus id: 0000:21:00.0, compute capability: 6.1)


### Setup TWILIO for text updates

In [2]:
account_sid = ''#config('ACCOUNT_ID',default='')
auth_token = ''#config('AUTHENTICATION_TOKEN',default='')
phone_num = ''#config('PHONE_NUMBER',default='')
client = Client(account_sid, auth_token)

#message = client.messages .create(
#                    body =  "Testing 1, 2, 3, 4", #Message you send
#                    from_ = '+12184383951',#Provided phone number
#                    to =    phone_num)#Your phone number
#message.sid

class SMSCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch%1 == 0:
            FP = logs['val_false_positives']
            TP = logs['val_true_positives']
            FN = logs['val_false_negatives']
            TN = logs['val_true_negatives']
            IOU_val = TP/(FN+TP+FP)
            message = client.messages .create(
                
                body = "For Epoch:{}, "
                        "Training loss={:.2f}, Validation loss ={:.2f} and Validation IOU = {:.2f}"
                .format(epoch, logs["loss"], logs["val_loss"], IOU_val),

                from_ = "+12184383951", to = phone_num)

            print(message.sid)

## Functions for reading in tfrecords

In [3]:
def parse_tfr_element(element):
    
    data = {
      'height': tf.io.FixedLenFeature([], tf.int64),
      'width':tf.io.FixedLenFeature([], tf.int64),
      'depth':tf.io.FixedLenFeature([], tf.int64),
      'raw_label':tf.io.FixedLenFeature([], tf.string),#tf.string = bytestring (not text string)
      'raw_image' : tf.io.FixedLenFeature([], tf.string),#tf.string = bytestring (not text string)
    }


    content = tf.io.parse_single_example(element, data)

    height = content['height']
    width = content['width']
    depth = content['depth']
    raw_label = content['raw_label']
    raw_image = content['raw_image']


    #get our 'feature'-- our image -- and reshape it appropriately
    feature = tf.io.parse_tensor(raw_image, out_type=tf.float16)
    feature = tf.reshape(feature, shape=[height,width,depth])
    label = tf.io.parse_tensor(raw_label, out_type=tf.int8)
    label = tf.reshape(label, shape=[height,width])
    return (feature, label)

def get_batched_dataset(filenames, testing = False):
    option_no_order = tf.data.Options()
    if testing:
        option_no_order.experimental_deterministic = True
    else:
        option_no_order.experimental_deterministic = False

    #dataset = tf.data.TFRecordDataset(filenames)
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.with_options(option_no_order)
    dataset = dataset.interleave(tf.data.TFRecordDataset, num_parallel_calls=AUTO)
    dataset = dataset.map(parse_tfr_element, num_parallel_calls=AUTO)

    #dataset = dataset.cache() # If dataset fits in RAM
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True) 
    if not testing:
        dataset = dataset.shuffle(2048)
    #dataset = dataset.repeat()
    
    dataset = dataset.prefetch(AUTO) #

    return dataset


def get_training_dataset(training_filenames):
    return get_batched_dataset(training_filenames)

def get_validation_dataset(validation_filenames):
    return get_batched_dataset(validation_filenames)

def get_test_dataset(test_filenames):
    return get_batched_dataset(test_filenames, testing = True)


## Loss functions

In [4]:
def FocalTverskyLoss(targets, inputs, alpha=1, beta=1, gamma=2, smooth=1e-6):
    '''
    ... in the case of α=β=0.5 the Tversky index simplifies to be 
    the same as the Dice coefficient, which is also equal to the F1 
    score. With α=β=1, Equation 2 produces Tanimoto coefficient, and 
    setting α+β=1 produces the set of Fβ scores. Larger βs weigh 
    recall higher than precision (by placing more emphasis on false negatives).
    '''
    #gamma>1 reduces the relative loss for well-classified examples 
    #alpha is a weighted term whose value is α for positive(foreground) alpha = 1 does nothing. alpha = 0.25 is best
    #class and 1-α for negative(background) class.
    #Larger βs weigh recall higher than precision (by placing more emphasis on false negatives)
    targets = tf.cast(targets,tf.float32)
    #flatten label and prediction tensors
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)

    #True Positives, False Positives & False Negatives
    TP = K.sum((inputs * targets))
    FP = K.sum(((1-targets) * inputs))
    FN = K.sum((targets * (1-inputs)))

    Tversky = (TP + smooth) / (TP + alpha*FP + beta*FN + smooth) 

    FocalTversky = K.pow((1 - Tversky), gamma)

    return FocalTversky

def IoULoss(targets, inputs, smooth=1e-6, gamma=3):
    targets = tf.cast(targets,tf.float32)
    #flatten label and prediction tensors
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)

    intersection = K.sum(K.dot(targets, inputs))
    total = K.sum(targets) + K.sum(inputs)
    union = total - intersection

    IoU = (intersection + smooth) / (union + smooth)
    return  K.pow((1 - IoU), gamma)

## Load Unet Model

In [5]:
unet = unet_model((IMG_W, IMG_H, N_CHANNELS),n_filters=N_FILTERS,n_classes=1)

2022-07-15 15:35:40.889380: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-15 15:35:40.889702: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:21:00.0 name: Quadro P2000 computeCapability: 6.1
coreClock: 1.4805GHz coreCount: 8 deviceMemorySize: 4.93GiB deviceMemoryBandwidth: 130.53GiB/s
2022-07-15 15:35:40.889740: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-07-15 15:35:40.889758: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-07-15 15:35:40.889769: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2022-07-15 15:35:40.889780: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2022-07

In [6]:
class ClearMemory(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        K.clear_session()

## Compile Unet and train Model

In [ ]:
unet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=[FocalTverskyLoss],
              metrics=[tfm.Precision(), tfm.Recall(), tfm.FalseNegatives(), tfm.FalsePositives(), tfm.TruePositives(), tfm.TrueNegatives()],
              run_eagerly=True)

unet.summary(line_length = 130)
print('mem usage: ', tf.config.experimental.get_memory_usage("GPU:0"))
print(get_model_memory_usage(BATCH_SIZE, unet))

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta = 0.01, patience=500, mode ="min", verbose = 2, restore_best_weights=True)
training_data = get_training_dataset(training_filenames)
validation_data = get_validation_dataset(validation_filenames)
#test_data = get_test_dataset(test_filenames)
print(training_data)
model_history = unet.fit(training_data, validation_data=validation_data, epochs=EPOCHS, callbacks = [SMSCallback(), ClearMemory(), callback])



Model: "model"
__________________________________________________________________________________________________________________________________
Layer (type)                              Output Shape                 Param #         Connected to                               
input_1 (InputLayer)                      [(None, 64, 64, 7)]          0                                                          
__________________________________________________________________________________________________________________________________
conv2d (Conv2D)                           (None, 64, 64, 32)           2048            input_1[0][0]                              
__________________________________________________________________________________________________________________________________
batch_normalization (BatchNormalization)  (None, 64, 64, 32)           128             conv2d[0][0]                               
____________________________________________________________________

2022-07-15 15:35:41.298460: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-07-15 15:35:41.310642: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3600000000 Hz
2022-07-15 15:35:44.110708: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-07-15 15:35:44.968294: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-07-15 15:35:45.010179: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-07-15 15:35:45.578610: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


39861/39861 [==============================] - 6282s 157ms/step - loss: 0.7565 - precision: 0.1552 - recall: 0.3269 - false_negatives: 35758099.8085 - false_positives: 83185177.0698 - true_positives: 17939304.4412 - true_negatives: 2475573514.9237 - val_loss: 0.7857 - val_precision: 0.2363 - val_recall: 0.3605 - val_false_negatives: 591711.0000 - val_false_positives: 1078058.0000 - val_true_positives: 333505.0000 - val_true_negatives: 433155904.0000
SM15d111f3ed1b4faaa4e71debcb95a36a
Epoch 2/100
39861/39861 [==============================] - 6264s 157ms/step - loss: 0.5065 - precision: 0.4927 - recall: 0.4509 - false_negatives: 30628246.5534 - false_positives: 25648460.3597 - true_positives: 25235154.7977 - true_negatives: 2530949551.7573 - val_loss: 0.7338 - val_precision: 0.2689 - val_recall: 0.4317 - val_false_negatives: 525771.0000 - val_false_positives: 1085915.0000 - val_true_positives: 399445.0000 - val_true_negatives: 433148544.0000
SM7c974760d8274c4f92d745bf95a38f1b
Epoch 3/10

## Save Unet results and weights

In [ ]:

fig, axs = plt.subplots(2,2, figsize=(15,15))

axs[0,0].plot(model_history.history['loss'])
axs[0,0].plot(model_history.history['val_loss'])
axs[0,0].set_title('model loss')
axs[0,0].set_ylabel('loss')

#axs[0].legend(['train', 'val'], loc='upper left')
#axs[0].show()





F1 = 2*np.divide(np.multiply(model_history.history['precision'],model_history.history['recall']), np.add(model_history.history['precision'], model_history.history['recall']))
F1_val = 2*np.divide(np.multiply(model_history.history['val_precision'],model_history.history['val_recall']), np.add(model_history.history['val_precision'], model_history.history['val_recall']))
FP = model_history.history['false_positives']
TP = model_history.history['true_positives']
FN = model_history.history['false_negatives']
TN = model_history.history['true_negatives']
IOU = np.divide(TP,np.add(FN,np.add(TP,FP)))
MCC = np.subtract(np.multiply(TP,TN),np.multiply(FP,FN))/np.sqrt(np.multiply(np.multiply(np.add(TP,FP),np.add(TP,FN)), np.multiply(np.add(TN,FP),np.add(TN,FN))))
FP = model_history.history['val_false_positives']
TP = model_history.history['val_true_positives']
FN = model_history.history['val_false_negatives']
TN = model_history.history['val_true_negatives']
MCC_val = np.subtract(np.multiply(TP,TN),np.multiply(FP,FN))/np.sqrt(np.multiply(np.multiply(np.add(TP,FP),np.add(TP,FN)), np.multiply(np.add(TN,FP),np.add(TN,FN))))
IOU_val = np.divide(TP,np.add(FN,np.add(TP,FP)))

axs[1,0].plot(IOU)
axs[1,0].plot(IOU_val)
axs[1,0].set_title(f'model IoU, max = {np.max(IOU_val)}')
axs[1,0].set_ylabel('IoU')
axs[1,0].legend(['train', 'val'], loc='upper right')

axs[0,1].plot(F1)
axs[0,1].plot(F1_val)
axs[0,1].set_title('model F1 score')
axs[0,1].set_ylabel('F1')
axs[0,1].set_xlabel('epoch')

axs[1,1].plot(MCC)
axs[1,1].plot(MCC_val)
axs[1,1].set_title('model MCC')
axs[1,1].set_ylabel('MCC')
axs[1,1].set_xlabel('epoch')


i = 1
while os.path.exists(Models_path+"Model" + VERSION + "run%s.png" % i):
    i += 1
plt.savefig(Models_path+"Model" + VERSION + "run%s.png" % i)

# Save the weights 
unet.save_weights(Models_path+ "Model" + VERSION + "run%s.ckpt" % i)

## Twilio end message

In [ ]:
 message = client.messages .create(
                
    body = "Model stopped at Epoch:{}, "
            "max Validation IOU = {:.2f}"
    .format(len(IOU_val), np.max(IOU_val)),

    from_ = "+12184383951", to = phone_num)

print(message.sid)